In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import gensim

from gensim.models import Word2Vec, KeyedVectors

from nltk.stem import WordNetLemmatizer , PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
import matplotlib.pyplot as plt
import nltk

from tensorflow.keras.utils import to_categorical

In [57]:
data = pd.read_csv("data/news_class.csv")

df = data.copy() # to be safe and avoid errors

df = df.loc[:,["data_id" , "content" , "category_level_1" , "category_level_2"]]
df.groupby(['category_level_1'])['data_id'].agg('count')

category_level_1
arts, culture, entertainment and media        300
conflict, war and peace                       800
crime, law and justice                        500
disaster, accident and emergency incident     500
economy, business and finance                 400
education                                     607
environment                                   600
health                                        700
human interest                                600
labour                                        703
lifestyle and leisure                         300
politics                                      900
religion and belief                           800
science and technology                        800
society                                      1100
sport                                         907
weather                                       400
Name: data_id, dtype: int64

In [58]:
def labeler(dataframe_column):
    encoder=LabelEncoder()
    
    labels = encoder.fit_transform(dataframe_column)
    print(encoder.classes_)
    print("We did it boys , labels have been created")
    
    return(pd.DataFrame(labels))

In [59]:
extra_stop = ['said', 'would','even','according','could','year','years','also','new','people','old,''one','two','time','first','last','say','make','best','get','three','make','year old','told','made','like','take','many','set','number','month','week','well','back' , 'post', 'http', 'www', 'presstv' , 'ir']
total_stop = stopwords.words("english") + extra_stop

# Function for removing ASCII characters
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

# Function for converting to lower case
def make_lower_case(text):
    return text.lower()

# Function for removing stop words
def remove_stop_words(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text =  tokenizer.tokenize(text)
    stops = total_stop
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

# Function for removing html
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

# Function for removing punctuation
def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

def lemm_text(text):
    lemm=WordNetLemmatizer()
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    tokens = text
    return ' '.join([lemm.lemmatize(t) for t in tokens])

def text_stemmer(text):
    stemmer=PorterStemmer()
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    tokens = text
    return ' '.join([stemmer.stem(t) for t in tokens])

In [61]:
df['Cleaned'] = df['content'].apply(_removeNonAscii)
df['Cleaned'] = df.Cleaned.apply(func = make_lower_case)
df['Cleaned'] = df.Cleaned.apply(func = remove_stop_words)
df['Cleaned'] = df.Cleaned.apply(func = remove_punctuation)
df['Cleaned'] = df.Cleaned.apply(func = remove_html)
df['Cleaned'] = df.Cleaned.apply(func = text_stemmer)
df = df.drop('content' , axis =1)

In [62]:
io = df.copy()

io = io.drop([374]).reset_index(drop=True)
io = io.drop([6527]).reset_index(drop=True)

In [64]:
corpus_full = []
for words in io['Cleaned']:
    corpus_full.append(words.split())  

In [65]:
def vectorizer():
    
    
    # Creating a list for storing the vectors ('Description' into vectors)
    #global word_embeddings
    word_embeddings = []
    i = 0
    # Reading the each 'Description'
    for line in io['Cleaned']:
        avgword2vec = None
        count = 0
        for word in line.split():
            if word in model.wv.key_to_index:
                count += 1
                if avgword2vec is None:
                    avgword2vec = model.wv[word]
                else:
                    avgword2vec = avgword2vec + model.wv[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
            word_embeddings.append(avgword2vec)
        else:
            print("I found it , the error occurs at line:" , i)
          
        i +=1 

    return(pd.DataFrame(word_embeddings))  # Returning our Data as a Dataframe (aesthetic reasons)

In [66]:
model = Word2Vec.load("word2vec.model3")

In [67]:
vect = vectorizer()

In [68]:
vect

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.001459,-0.078473,-0.092210,0.052347,0.102376,-0.002754,-0.056390,0.037248,-0.053454,0.207812,...,-0.058640,0.173090,-0.018010,0.045516,0.258875,0.031823,-0.007939,-0.085078,0.036334,-0.109643
1,0.072767,-0.063284,-0.092942,0.015908,0.083422,0.004476,-0.067774,0.043507,-0.000405,0.186417,...,-0.022937,0.108182,-0.009431,0.064129,0.210500,0.040490,-0.036917,-0.046180,-0.024272,-0.064851
2,0.040674,0.019691,-0.046176,0.082028,0.135475,-0.008119,-0.007749,0.072441,-0.066983,0.221950,...,-0.025967,0.072008,-0.051875,0.114976,0.170444,0.098256,-0.050679,-0.022611,-0.050753,-0.058318
3,0.100426,-0.135048,-0.067858,0.048585,0.099610,0.015120,-0.076442,0.033414,-0.030683,0.204774,...,-0.048312,0.127591,-0.027277,0.057416,0.185428,0.101191,-0.045935,-0.063575,-0.013838,-0.118549
4,0.040711,-0.018188,-0.070330,0.057477,0.106131,-0.021223,-0.070114,0.045974,-0.020043,0.177833,...,-0.067081,0.080117,-0.014415,0.098188,0.157175,0.078036,-0.014283,-0.087883,-0.085483,-0.048128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10910,0.072800,-0.021963,-0.076825,-0.089859,0.062972,-0.039061,-0.005116,-0.047534,-0.084299,0.020017,...,-0.125978,0.118477,-0.050200,0.117858,0.248817,0.005598,-0.001825,-0.123827,0.012053,-0.021778
10911,0.114847,-0.079585,-0.117073,-0.018464,0.158374,-0.127964,-0.093952,-0.101651,-0.051418,0.090127,...,-0.107329,0.046495,0.006817,0.071641,0.254237,0.066662,-0.049829,-0.263125,-0.038419,0.042634
10912,0.109188,0.045075,-0.084913,-0.135421,0.069145,0.031762,-0.064714,0.009175,-0.052627,0.014560,...,0.008871,0.046193,0.000713,0.234333,0.179953,0.029830,-0.012984,-0.046922,0.042794,-0.046787
10913,0.035525,-0.023847,-0.118682,-0.085456,0.090627,0.014579,-0.010420,0.026654,-0.094493,0.038901,...,-0.008083,0.086267,0.029471,0.199954,0.192337,0.017063,-0.037731,-0.130126,0.075293,-0.019874


In [13]:
scaler = MinMaxScaler()

scaler.fit(vect)

vect_trans = scaler.transform(vect)

In [14]:
vect_trans

array([[0.4599773 , 0.5001998 , 0.52529293, ..., 0.4052878 , 0.5324256 ,
        0.27569968],
       [0.58168113, 0.47737575, 0.45707464, ..., 0.46718958, 0.4704046 ,
        0.45043117],
       [0.50672764, 0.6408195 , 0.54765874, ..., 0.5150039 , 0.45378554,
        0.41897237],
       ...,
       [0.5929465 , 0.5836759 , 0.50009054, ..., 0.45045012, 0.60331523,
        0.40922025],
       [0.47044164, 0.4418217 , 0.43300474, ..., 0.31804413, 0.6307093 ,
        0.43886894],
       [0.54568547, 0.40901166, 0.44107127, ..., 0.32701153, 0.504436  ,
        0.4860566 ]], dtype=float32)

In [69]:
y1 = labeler(io["category_level_1"])


X = vect.copy()

['arts, culture, entertainment and media' 'conflict, war and peace'
 'crime, law and justice' 'disaster, accident and emergency incident'
 'economy, business and finance' 'education' 'environment' 'health'
 'human interest' 'labour' 'lifestyle and leisure' 'politics'
 'religion and belief' 'science and technology' 'society' 'sport'
 'weather']
We did it boys , labels have been created


In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.2,random_state=42)

In [71]:
y_train1 = to_categorical(y_train)

In [72]:
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.layers import Flatten , Input , Conv2D , MaxPooling2D , BatchNormalization

In [93]:
cnn_model = tf.keras.Sequential()
cnn_model.add(tf.keras.layers.Conv1D(256,3,input_shape=(200,1), activation='leaky_relu'))
#cnn_model.add(tf.keras.layers.MaxPool1D())
cnn_model.add(tf.keras.layers.Conv1D(128,3,activation='leaky_relu'))
#cnn_model.add(tf.keras.layers.MaxPool1D())
cnn_model.add(tf.keras.layers.Conv1D(64,3,activation='leaky_relu')) 
cnn_model.add(tf.keras.layers.MaxPool1D())
cnn_model.add(tf.keras.layers.BatchNormalization())
cnn_model.add(tf.keras.layers.Flatten())
cnn_model.add(tf.keras.layers.Dense(64,activation='leaky_relu'))
cnn_model.add(tf.keras.layers.Dense(30,activation='leaky_relu'))
cnn_model.add(tf.keras.layers.Dense(17,activation='softmax'))

In [94]:
cnn_model.compile(optimizer='adam', 
          loss = 'categorical_crossentropy',
          metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                   tf.keras.metrics.Precision(name='precision') ,
                   tf.keras.metrics.Recall(name='recall')])

In [95]:
cnn_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 198, 256)          1024      
                                                                 
 conv1d_4 (Conv1D)           (None, 196, 128)          98432     
                                                                 
 conv1d_5 (Conv1D)           (None, 194, 64)           24640     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 97, 64)            0         
 g1D)                                                            
                                                                 
 batch_normalization_1 (Bat  (None, 97, 64)            256       
 chNormalization)                                                
                                                                 
 flatten_1 (Flatten)         (None, 6208)            

In [96]:
cnn_model.fit(X_train, y_train1, batch_size = 16 ,epochs=2, validation_split=(0.2) ,verbose =1)

Epoch 1/2
437/437 [==============================] - 15s 30ms/step - loss: 1.5424 - accuracy: 0.9535 - precision: 0.7200 - recall: 0.3432 - val_loss: 1.8982 - val_accuracy: 0.9416 - val_precision: 0.9286 - val_recall: 0.0074
Epoch 2/2
437/437 [==============================] - 13s 30ms/step - loss: 1.1716 - accuracy: 0.9607 - precision: 0.7518 - recall: 0.4948 - val_loss: 1.2370 - val_accuracy: 0.9600 - val_precision: 0.7672 - val_recall: 0.4585


In [97]:

predictions = np.argmax(cnn_model.predict(X_test), axis=-1)

print(classification_report(y_test.values , predictions))

69/69 [==============================] - 1s 14ms/step
              precision    recall  f1-score   support

           0       0.67      0.32      0.44        68
           1       0.69      0.75      0.72       176
           2       0.45      0.89      0.59        97
           3       0.56      0.78      0.65        90
           4       0.67      0.48      0.56        93
           5       0.68      0.67      0.67       108
           6       0.90      0.52      0.66       126
           7       0.72      0.68      0.70       136
           8       0.74      0.45      0.56       122
           9       0.76      0.64      0.69       155
          10       0.41      0.79      0.54        61
          11       0.44      0.67      0.53       172
          12       0.74      0.68      0.71       182
          13       0.59      0.56      0.57       151
          14       0.56      0.34      0.42       200
          15       0.76      0.82      0.79       169
          16       0.90    

In [98]:
for i in range(100):
    
    print( "At iteration "  ,i+2 , " we get these data :" )
    
    cnn_model.fit(X_train , y_train1 , batch_size = 64 , epochs = 1 , validation_split=(0.2) ,verbose =1)

    predictions = np.argmax(cnn_model.predict(X_test), axis=-1)

    print(classification_report(y_test.values , predictions))

At iteration  2  we get these data :
69/69 [==============================] - 1s 13ms/step
              precision    recall  f1-score   support

           0       0.61      0.41      0.49        68
           1       0.84      0.58      0.69       176
           2       0.73      0.60      0.66        97
           3       0.78      0.58      0.66        90
           4       0.77      0.52      0.62        93
           5       0.66      0.64      0.65       108
           6       0.80      0.78      0.79       126
           7       0.69      0.77      0.73       136
           8       0.59      0.74      0.65       122
           9       0.80      0.54      0.65       155
          10       0.67      0.57      0.62        61
          11       0.51      0.65      0.57       172
          12       0.70      0.72      0.71       182
          13       0.55      0.63      0.59       151
          14       0.43      0.60      0.50       200
          15       0.83      0.82      0.82 

69/69 [==============================] - 1s 15ms/step
              precision    recall  f1-score   support

           0       0.64      0.34      0.44        68
           1       0.79      0.72      0.75       176
           2       0.73      0.57      0.64        97
           3       0.62      0.61      0.62        90
           4       0.76      0.42      0.54        93
           5       0.68      0.64      0.66       108
           6       0.81      0.72      0.76       126
           7       0.54      0.84      0.65       136
           8       0.64      0.64      0.64       122
           9       0.84      0.49      0.62       155
          10       0.61      0.72      0.66        61
          11       0.46      0.67      0.55       172
          12       0.70      0.74      0.72       182
          13       0.54      0.63      0.58       151
          14       0.54      0.46      0.50       200
          15       0.79      0.82      0.80       169
          16       0.78    

69/69 [==============================] - 1s 14ms/step
              precision    recall  f1-score   support

           0       0.65      0.41      0.50        68
           1       0.77      0.70      0.73       176
           2       0.80      0.57      0.66        97
           3       0.61      0.69      0.65        90
           4       0.70      0.48      0.57        93
           5       0.59      0.68      0.63       108
           6       0.82      0.64      0.72       126
           7       0.64      0.78      0.70       136
           8       0.66      0.69      0.67       122
           9       0.80      0.60      0.69       155
          10       0.82      0.67      0.74        61
          11       0.49      0.73      0.58       172
          12       0.65      0.75      0.70       182
          13       0.56      0.58      0.57       151
          14       0.62      0.52      0.57       200
          15       0.77      0.89      0.83       169
          16       0.89    

KeyboardInterrupt: 

In [21]:
cnn_model.fit(X_train, y_train1, batch_size = 16 ,epochs=3, validation_split=(0.2) ,verbose =1)

Epoch 1/3
437/437 [==============================] - 12s 28ms/step - loss: 0.9620 - accuracy: 0.9662 - precision: 0.7791 - recall: 0.5941 - val_loss: 1.2586 - val_accuracy: 0.9610 - val_precision: 0.7159 - val_recall: 0.5581
Epoch 2/3
437/437 [==============================] - 11s 26ms/step - loss: 0.8831 - accuracy: 0.9685 - precision: 0.7934 - recall: 0.6289 - val_loss: 1.1667 - val_accuracy: 0.9626 - val_precision: 0.7356 - val_recall: 0.5684
Epoch 3/3
437/437 [==============================] - 12s 26ms/step - loss: 0.8455 - accuracy: 0.9694 - precision: 0.7982 - recall: 0.6432 - val_loss: 1.1566 - val_accuracy: 0.9634 - val_precision: 0.7436 - val_recall: 0.5776


In [22]:
predictions = np.argmax(cnn_model.predict(X_test), axis=-1)

print(classification_report(y_test.values , predictions))

69/69 [==============================] - 1s 14ms/step
              precision    recall  f1-score   support

           0       0.46      0.46      0.46        68
           1       0.78      0.69      0.73       176
           2       0.55      0.80      0.65        97
           3       0.52      0.69      0.59        90
           4       0.70      0.55      0.61        93
           5       0.67      0.69      0.68       108
           6       0.77      0.79      0.78       126
           7       0.59      0.88      0.71       136
           8       0.67      0.43      0.53       122
           9       0.85      0.63      0.73       155
          10       0.90      0.44      0.59        61
          11       0.72      0.49      0.58       172
          12       0.76      0.75      0.75       182
          13       0.42      0.66      0.51       151
          14       0.62      0.41      0.50       200
          15       0.78      0.92      0.85       169
          16       0.82    

In [23]:
cnn_model.fit(X_train, y_train1, batch_size = 16 ,epochs=1, validation_split=(0.2) ,verbose =1)

predictions = np.argmax(cnn_model.predict(X_test), axis=-1)

print(classification_report(y_test.values , predictions))

69/69 [==============================] - 1s 14ms/step
              precision    recall  f1-score   support

           0       0.89      0.25      0.39        68
           1       0.83      0.69      0.75       176
           2       0.69      0.65      0.67        97
           3       0.88      0.49      0.63        90
           4       0.51      0.63      0.56        93
           5       0.62      0.83      0.71       108
           6       0.87      0.71      0.78       126
           7       0.62      0.82      0.70       136
           8       0.42      0.84      0.56       122
           9       0.86      0.62      0.72       155
          10       0.71      0.44      0.55        61
          11       0.65      0.52      0.58       172
          12       0.74      0.72      0.73       182
          13       0.80      0.40      0.54       151
          14       0.45      0.62      0.52       200
          15       0.85      0.83      0.84       169
          16       0.71    

In [46]:
# 6 ---> 69%
# 7 ---> 70%
# 8 ---> 67%
# 9 ---> 71
# 10 ---> 64%
#11 ---> 69%
#12 ---> 69%
#13 ---> 69%
# 14 ---> 70 %
# 15 ---> 71 %
# 16 ---> 69 %
# 17 --- > 71 %
# 18 ---> 70 %
# 19 ---> 70 %
# 20 ---> 70 %

# MODEL FC

In [86]:
model6 = Sequential()
model6.add(Dense(100,input_dim=200,activation = "leaky_relu"))
model6.add(Dense(180,activation = "leaky_relu"))
model6.add(tf.keras.layers.Dropout(0.2))
model6.add(Dense(60,activation = "leaky_relu"))
#model6.add(Dense(30,activation = "leaky_relu"))
model6.add(Dense(17,activation = "softmax"))

In [87]:
opt = tf.keras.optimizers.AdamW(learning_rate =0.007 , beta_1=0.9,
    beta_2=0.999,
    use_ema=True,
    ema_momentum=0.99)

In [88]:
model6.compile(optimizer = opt , 
              loss = 'categorical_crossentropy' ,
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

In [89]:
model6.fit(X_train , y_train1 , batch_size = 16 , epochs = 5 , validation_split=(0.2) ,verbose =1)

Epoch 1/5
437/437 [==============================] - 3s 3ms/step - loss: 1.6885 - accuracy: 0.9505 - precision: 0.7043 - recall: 0.2739 - val_loss: 1.3865 - val_accuracy: 0.9579 - val_precision: 0.7616 - val_recall: 0.4133
Epoch 2/5
437/437 [==============================] - 1s 3ms/step - loss: 1.3686 - accuracy: 0.9561 - precision: 0.7225 - recall: 0.4116 - val_loss: 1.4017 - val_accuracy: 0.9575 - val_precision: 0.7366 - val_recall: 0.4322
Epoch 3/5
437/437 [==============================] - 1s 3ms/step - loss: 1.2964 - accuracy: 0.9578 - precision: 0.7319 - recall: 0.4451 - val_loss: 1.3009 - val_accuracy: 0.9598 - val_precision: 0.7447 - val_recall: 0.4825
Epoch 4/5
437/437 [==============================] - 1s 3ms/step - loss: 1.2337 - accuracy: 0.9595 - precision: 0.7371 - recall: 0.4829 - val_loss: 1.3808 - val_accuracy: 0.9572 - val_precision: 0.7070 - val_recall: 0.4654
Epoch 5/5
437/437 [==============================] - 1s 3ms/step - loss: 1.1781 - accuracy: 0.9609 - precisi

In [90]:
predictions = np.argmax(model6.predict(X_test), axis=-1)

predictions

69/69 [==============================] - 0s 2ms/step


array([14,  1, 16, ..., 11,  9, 12], dtype=int64)

In [91]:
print(classification_report(y_test.values , predictions))

              precision    recall  f1-score   support

           0       0.50      0.50      0.50        68
           1       0.77      0.72      0.74       176
           2       0.69      0.78      0.73        97
           3       0.76      0.61      0.68        90
           4       0.82      0.43      0.56        93
           5       0.66      0.67      0.66       108
           6       0.73      0.84      0.78       126
           7       0.69      0.70      0.70       136
           8       0.78      0.44      0.57       122
           9       0.78      0.63      0.70       155
          10       0.55      0.66      0.60        61
          11       0.49      0.66      0.56       172
          12       0.75      0.75      0.75       182
          13       0.73      0.50      0.60       151
          14       0.46      0.60      0.52       200
          15       0.76      0.84      0.80       169
          16       0.80      0.96      0.88        77

    accuracy              

In [92]:
for i in range(100):
    
    print( "At iteration "  ,i+6 , " we get these data :" )
    
    model6.fit(X_train , y_train1 , batch_size = 64 , epochs = 1 , validation_split=(0.2) ,verbose =1)

    predictions = np.argmax(model6.predict(X_test), axis=-1)

    print(classification_report(y_test.values , predictions))

At iteration  6  we get these data :
69/69 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.49      0.46      0.47        68
           1       0.78      0.73      0.75       176
           2       0.68      0.74      0.71        97
           3       0.72      0.61      0.66        90
           4       0.82      0.45      0.58        93
           5       0.65      0.67      0.66       108
           6       0.75      0.83      0.79       126
           7       0.70      0.71      0.70       136
           8       0.73      0.52      0.61       122
           9       0.77      0.63      0.69       155
          10       0.61      0.66      0.63        61
          11       0.50      0.64      0.56       172
          12       0.74      0.76      0.75       182
          13       0.69      0.54      0.61       151
          14       0.49      0.62      0.55       200
          15       0.78      0.86      0.81  

69/69 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.55      0.47      0.51        68
           1       0.80      0.74      0.77       176
           2       0.68      0.75      0.71        97
           3       0.72      0.62      0.67        90
           4       0.77      0.51      0.61        93
           5       0.64      0.70      0.67       108
           6       0.79      0.83      0.81       126
           7       0.71      0.70      0.71       136
           8       0.72      0.60      0.65       122
           9       0.77      0.66      0.71       155
          10       0.67      0.66      0.66        61
          11       0.55      0.65      0.60       172
          12       0.71      0.75      0.73       182
          13       0.66      0.59      0.62       151
          14       0.52      0.60      0.56       200
          15       0.78      0.86      0.82       169
          16       0.81     

69/69 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.55      0.49      0.52        68
           1       0.80      0.75      0.77       176
           2       0.72      0.75      0.73        97
           3       0.69      0.60      0.64        90
           4       0.80      0.55      0.65        93
           5       0.66      0.72      0.69       108
           6       0.80      0.80      0.80       126
           7       0.69      0.70      0.69       136
           8       0.75      0.62      0.68       122
           9       0.78      0.66      0.72       155
          10       0.72      0.67      0.69        61
          11       0.59      0.68      0.63       172
          12       0.72      0.76      0.74       182
          13       0.65      0.58      0.61       151
          14       0.53      0.64      0.58       200
          15       0.78      0.86      0.82       169
          16       0.80     

69/69 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.57      0.50      0.53        68
           1       0.78      0.74      0.76       176
           2       0.73      0.75      0.74        97
           3       0.70      0.63      0.67        90
           4       0.82      0.54      0.65        93
           5       0.65      0.75      0.70       108
           6       0.82      0.87      0.84       126
           7       0.73      0.74      0.74       136
           8       0.72      0.61      0.66       122
           9       0.80      0.69      0.74       155
          10       0.68      0.62      0.65        61
          11       0.58      0.66      0.62       172
          12       0.72      0.75      0.73       182
          13       0.65      0.58      0.61       151
          14       0.56      0.62      0.59       200
          15       0.78      0.87      0.82       169
          16       0.82     

69/69 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.48      0.51      0.50        68
           1       0.78      0.77      0.78       176
           2       0.71      0.72      0.72        97
           3       0.69      0.63      0.66        90
           4       0.78      0.53      0.63        93
           5       0.65      0.74      0.69       108
           6       0.82      0.84      0.83       126
           7       0.73      0.73      0.73       136
           8       0.71      0.60      0.65       122
           9       0.78      0.67      0.72       155
          10       0.67      0.64      0.66        61
          11       0.59      0.65      0.62       172
          12       0.74      0.75      0.74       182
          13       0.66      0.60      0.63       151
          14       0.56      0.62      0.59       200
          15       0.80      0.86      0.83       169
          16       0.82     

...

69/69 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.56      0.57      0.57        68
           1       0.82      0.74      0.78       176
           2       0.72      0.64      0.68        97
           3       0.64      0.62      0.63        90
           4       0.76      0.55      0.64        93
           5       0.59      0.69      0.64       108
           6       0.79      0.79      0.79       126
           7       0.74      0.74      0.74       136
           8       0.68      0.62      0.65       122
           9       0.81      0.70      0.75       155
          10       0.71      0.74      0.73        61
          11       0.56      0.63      0.60       172
          12       0.70      0.73      0.72       182
          13       0.62      0.63      0.62       151
          14       0.59      0.62      0.60       200
          15       0.84      0.89      0.86       169
          16       0.83     

69/69 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.62      0.57      0.60        68
           1       0.80      0.74      0.77       176
           2       0.70      0.67      0.68        97
           3       0.66      0.62      0.64        90
           4       0.74      0.55      0.63        93
           5       0.61      0.69      0.65       108
           6       0.80      0.79      0.80       126
           7       0.75      0.73      0.74       136
           8       0.64      0.63      0.63       122
           9       0.79      0.70      0.74       155
          10       0.72      0.69      0.71        61
          11       0.59      0.64      0.62       172
          12       0.70      0.75      0.73       182
          13       0.64      0.63      0.63       151
          14       0.60      0.65      0.62       200
          15       0.82      0.89      0.85       169
          16       0.83     

69/69 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.59      0.57      0.58        68
           1       0.81      0.76      0.78       176
           2       0.69      0.70      0.70        97
           3       0.67      0.63      0.65        90
           4       0.78      0.57      0.66        93
           5       0.58      0.69      0.63       108
           6       0.82      0.81      0.82       126
           7       0.74      0.71      0.73       136
           8       0.68      0.65      0.66       122
           9       0.77      0.68      0.72       155
          10       0.68      0.70      0.69        61
          11       0.58      0.60      0.59       172
          12       0.72      0.72      0.72       182
          13       0.62      0.64      0.63       151
          14       0.59      0.64      0.61       200
          15       0.82      0.87      0.84       169
          16       0.83     

69/69 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.59      0.57      0.58        68
           1       0.79      0.74      0.77       176
           2       0.69      0.68      0.69        97
           3       0.65      0.60      0.62        90
           4       0.76      0.55      0.64        93
           5       0.60      0.68      0.64       108
           6       0.79      0.76      0.77       126
           7       0.75      0.74      0.74       136
           8       0.66      0.63      0.65       122
           9       0.77      0.66      0.71       155
          10       0.68      0.66      0.67        61
          11       0.58      0.65      0.61       172
          12       0.72      0.73      0.72       182
          13       0.64      0.62      0.63       151
          14       0.58      0.65      0.61       200
          15       0.80      0.88      0.84       169
          16       0.83     

69/69 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.60      0.57      0.59        68
           1       0.80      0.76      0.78       176
           2       0.69      0.68      0.69        97
           3       0.65      0.60      0.62        90
           4       0.74      0.57      0.64        93
           5       0.59      0.69      0.63       108
           6       0.77      0.75      0.76       126
           7       0.77      0.75      0.76       136
           8       0.64      0.61      0.62       122
           9       0.80      0.68      0.74       155
          10       0.71      0.75      0.73        61
          11       0.56      0.62      0.59       172
          12       0.71      0.72      0.72       182
          13       0.63      0.59      0.61       151
          14       0.57      0.64      0.61       200
          15       0.84      0.88      0.86       169
          16       0.82     

# MODEL 2

In [41]:
model2 = Sequential()
model2.add(Dense(100,input_dim=200,activation = "leaky_relu"))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(Dense(80,activation = "leaky_relu"))
model2.add(Dense(40,activation = "leaky_relu"))
model2.add(Dense(25,activation = "leaky_relu"))
model2.add(Dense(17,activation = "softmax"))

In [42]:
opt = tf.keras.optimizers.AdamW(learning_rate =0.007 , beta_1=0.9,
    beta_2=0.999,
    use_ema=True,
    ema_momentum=0.99)

In [43]:
model2.compile(optimizer = opt , 
              loss = 'categorical_crossentropy' ,
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

In [44]:
for i in range(100):
    
    print( "At iteration "  ,i , " we get these data :" )
    
    model2.fit(X_train , y_train1 , batch_size = 32 , epochs = 1 , validation_split=(0.2) ,verbose =1)

    predictions = np.argmax(model2.predict(X_test), axis=-1)

    print(classification_report(y_test.values , predictions))

At iteration  0  we get these data :
69/69 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.70      0.10      0.18        68
           1       0.66      0.72      0.69       176
           2       0.79      0.56      0.65        97
           3       0.80      0.50      0.62        90
           4       0.70      0.42      0.52        93
           5       0.66      0.66      0.66       108
           6       0.71      0.81      0.76       126
           7       0.63      0.74      0.68       136
           8       0.41      0.32      0.36       122
           9       0.70      0.60      0.65       155
          10       0.78      0.23      0.35        61
          11       0.45      0.58      0.51       172
          12       0.61      0.76      0.67       182
          13       0.54      0.60      0.56       151
          14       0.40      0.41      0.41       200
          15       0.72      0.94      0.81  

69/69 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.54      0.43      0.48        68
           1       0.73      0.73      0.73       176
           2       0.69      0.78      0.73        97
           3       0.79      0.63      0.70        90
           4       0.74      0.53      0.62        93
           5       0.68      0.75      0.71       108
           6       0.78      0.83      0.80       126
           7       0.68      0.79      0.73       136
           8       0.69      0.46      0.55       122
           9       0.82      0.65      0.73       155
          10       0.57      0.66      0.61        61
          11       0.53      0.56      0.54       172
          12       0.73      0.80      0.76       182
          13       0.63      0.61      0.62       151
          14       0.52      0.56      0.54       200
          15       0.78      0.86      0.82       169
          16       0.83     

69/69 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.56      0.50      0.53        68
           1       0.77      0.76      0.76       176
           2       0.73      0.74      0.74        97
           3       0.74      0.61      0.67        90
           4       0.70      0.55      0.61        93
           5       0.69      0.79      0.74       108
           6       0.79      0.83      0.81       126
           7       0.72      0.71      0.71       136
           8       0.70      0.57      0.63       122
           9       0.81      0.63      0.71       155
          10       0.62      0.62      0.62        61
          11       0.55      0.62      0.58       172
          12       0.73      0.77      0.75       182
          13       0.66      0.64      0.65       151
          14       0.58      0.64      0.61       200
          15       0.78      0.88      0.83       169
          16       0.81     

69/69 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.58      0.53      0.55        68
           1       0.76      0.79      0.77       176
           2       0.71      0.75      0.73        97
           3       0.77      0.64      0.70        90
           4       0.73      0.56      0.63        93
           5       0.67      0.77      0.72       108
           6       0.83      0.83      0.83       126
           7       0.71      0.74      0.72       136
           8       0.75      0.61      0.67       122
           9       0.81      0.67      0.73       155
          10       0.68      0.72      0.70        61
          11       0.60      0.66      0.63       172
          12       0.77      0.81      0.79       182
          13       0.69      0.61      0.65       151
          14       0.58      0.62      0.60       200
          15       0.78      0.86      0.82       169
          16       0.82     

69/69 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.58      0.54      0.56        68
           1       0.76      0.77      0.77       176
           2       0.76      0.76      0.76        97
           3       0.74      0.63      0.68        90
           4       0.69      0.58      0.63        93
           5       0.68      0.80      0.73       108
           6       0.83      0.83      0.83       126
           7       0.71      0.77      0.74       136
           8       0.78      0.61      0.68       122
           9       0.80      0.64      0.71       155
          10       0.65      0.72      0.68        61
          11       0.62      0.60      0.61       172
          12       0.73      0.79      0.76       182
          13       0.68      0.63      0.65       151
          14       0.57      0.63      0.60       200
          15       0.80      0.86      0.83       169
          16       0.82     

69/69 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.57      0.57      0.57        68
           1       0.79      0.78      0.79       176
           2       0.71      0.79      0.75        97
           3       0.70      0.63      0.67        90
           4       0.68      0.59      0.63        93
           5       0.63      0.80      0.70       108
           6       0.84      0.86      0.85       126
           7       0.75      0.74      0.74       136
           8       0.72      0.61      0.66       122
           9       0.79      0.67      0.73       155
          10       0.71      0.66      0.68        61
          11       0.62      0.59      0.60       172
          12       0.73      0.82      0.77       182
          13       0.65      0.64      0.64       151
          14       0.64      0.59      0.62       200
          15       0.80      0.87      0.83       169
          16       0.82     

69/69 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.57      0.56      0.56        68
           1       0.77      0.80      0.78       176
           2       0.71      0.77      0.74        97
           3       0.72      0.64      0.68        90
           4       0.69      0.58      0.63        93
           5       0.69      0.74      0.71       108
           6       0.87      0.82      0.84       126
           7       0.74      0.77      0.76       136
           8       0.76      0.69      0.72       122
           9       0.82      0.66      0.73       155
          10       0.74      0.74      0.74        61
          11       0.63      0.56      0.59       172
          12       0.71      0.81      0.76       182
          13       0.70      0.62      0.65       151
          14       0.60      0.67      0.63       200
          15       0.80      0.89      0.84       169
          16       0.82     

197/219 [=========================>....] - ETA: 0s - loss: 0.7144 - accuracy: 0.9740 - precision: 0.8261 - recall: 0.7061

KeyboardInterrupt: 

In [62]:
stopwords.words("english")

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [64]:
extra_stop

['said',
 'would',
 'even',
 'according',
 'could',
 'year',
 'years',
 'also',
 'new',
 'people',
 'old,one',
 'two',
 'time',
 'first',
 'last',
 'say',
 'make',
 'best',
 'get',
 'three',
 'make',
 'year old',
 'told',
 'made',
 'like',
 'take',
 'many',
 'set',
 'number',
 'month',
 'week',
 'well',
 'back']

In [63]:
io

,data_id,category_level_1,category_level_2,Cleaned
0,1809,"crime, law and justice",crime,virginia woman whose 2 old son found trash inc...
1,1980,"crime, law and justice",crime,author tri determin anyon help inmat escap cal...
2,1995,"crime, law and justice",crime,13 old suspect doubl homicid escap custodi ret...
3,2740,"crime, law and justice",crime,mother young children found hang pennsylvania ...
4,7038,"crime, law and justice",crime,one famili member derek violent attack rex pas...
...,...,...,...,...
10910,907640,"conflict, war and peace",post-war reconstruction,origin publish site beirut lebanon 10 30 deput...
10911,892720,"conflict, war and peace",post-war reconstruction,origin publish site kiev octob 12 tass david a...
10912,870499,"conflict, war and peace",post-war reconstruction,detail 2019 07 06 600319 iran support iraq rec...
10913,887334,"conflict, war and peace",post-war reconstruction,detail 2019 09 25 607114 iraq salih terror ori...


In [76]:
total_stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each